# **함께누리몰 목록 크롤링**
- 판매상품 크롤링
- 판매자 크롤링

## **1 판매상품 정보 수집기**

In [100]:
# https://wikidocs.net/48470
# https://www.hknuri.co.kr/shop/about/seller
import requests
import pandas as pd
from tqdm import tqdm
from lxml.html import fromstring, tostring

def get_page_link(pages=405):
    link_data = []
    for page in tqdm(range(pages)):
        url = f"https://www.hknuri.co.kr/shop/jachigu/00?authCategory=,&pageNum={page}"
        response = requests.get(url).text
        response_lxml = fromstring(response)
        items = response_lxml.xpath('//div[@class="ch-container"]/ul/li')

        # Url Links
        for item in items:
            link = "https://www.hknuri.co.kr" + item.xpath('.//div[@class="name"]/a')[0].get("href")
            title = item.xpath('.//div[@class="name"]/a')[0].text
            link_data.append([link, title])
    return link_data

def get_item_text(url):
    response = requests.get(url).text
    response_lxml = fromstring(response)
    table = response_lxml.xpath('//div[@class="ch-detail-box"]//table')
    try:
        table = tostring(table[0], encoding=str)
        table = pd.read_html(table)[0]
        return table[1].tolist()
    except:
        return None

In [101]:
file_name = 'data/hnm_page.csv'
# links = get_page_link()
# table = pd.DataFrame(links)
# table.columns = ['link', 'title']
# table.to_csv('data/hnm_page.csv', index=None)
items = pd.read_csv(file_name)
items.tail(3)

,link,title
12111,https://www.hknuri.co.kr/shop/detail/7,해다미 복분자원
12112,https://www.hknuri.co.kr/shop/detail/4,[어둠속의대화] 촉각노트
12113,https://www.hknuri.co.kr/shop/detail/3,[어둠속의대화] 연필


In [102]:
file_name = 'data/hnm_items.csv'
from tqdm import tqdm

# result = []
# for url in tqdm(items['link']):
#     result_list = get_item_text(url)
#     if result_list:
#         result.append(result_list)

# table = pd.DataFrame(result)
# table.columns = ['상품', '업체', "구가격", "수량", "옵션", "배송비", "상품코드", "인증", "가격"]
# table = table.loc[:, ['업체','상품',"인증","가격","상품코드"]]
# table.to_csv(file_name, index=None)
table = pd.read_csv(file_name)
table.tail(3)

,업체,상품,인증,가격,상품코드
12104,사회복지법인 다운회 아름다운,해다미 복분자원,사회적기업 중증장애인생산품,"24,000 원",7
12105,(주)엔비전스,[어둠속의대화] 촉각노트,사회적기업,"6,000 원",4
12106,(주)엔비전스,[어둠속의대화] 연필,사회적기업,"1,000 원",3


## **2 판매사 정보 수집**
### **01 가게명 및 URL 수집**

In [103]:
def get_market_link(page=34):
    url = f"https://www.hknuri.co.kr/shop/about/seller?pageNum={page}&search=name&findword="
    response = requests.get(url).text
    response_lxml = fromstring(response)
    items = response_lxml.xpath('//div[@class="row"]/div[@class="col-md-6"]')
    result = []
    for item in items:
        link_url = item.get("onclick").split('=')[-1].replace("'", "")
        title = item.xpath(".//strong/text()")[0]
        result.append([link_url, title])
    return result

result = []
# from tqdm import tqdm
# for _ in tqdm(range(34)):
#     result += get_market_link(_)

# print(len(result))
# table = pd.DataFrame(result)
# table.columns = ['url', 'name']
# table.to_csv(file_name, index=None)

In [109]:
import pandas as pd
file_name = 'data/hnm_store_url.csv'
table = pd.read_csv(file_name)
table.tail(3)

,url,name
638,/shop/about/seller/2458,(주)심원테크
639,/shop/about/seller/2457,(주)프레시플라워
640,/shop/about/seller/2456,아이티희망나눔(주)


### **02 가게 상세정보 수집**

In [114]:
def get_company_info(name, sub_url):
    result = {}
    result['name'] = name
    url = f'https://www.hknuri.co.kr{sub_url}'
    response = requests.get(url).text
    response_lxml = fromstring(response)
    response_lxml.xpath('//table//td[@class="seller-table-bottom"]')[0]
    items = [item.xpath('.//text()') for item in response_lxml.xpath('.//div[@class="seller-info-box"]')[:6]]

    for item in items:
        texts = "".join(item).strip().replace(u'\xa0', u' ')
        texts = texts.split('\r\n\t\t\t\t\t')
        result[texts[0]] = "".join(texts[1:])
    return result

In [115]:
file_name = 'data/hnm_store.csv'

# result = []
# for no in tqdm(range(table.shape[0])):
#     _table = table.iloc[no, :]
#     sub_url = _table[0]
#     name    = _table[1]
#     infos   = get_company_info(name, sub_url)
#     result.append(infos)

# table = pd.DataFrame(result)
# table.to_csv(file_name, index=None)
# table = pd.read_csv(file_name).fillna('')
# table.columns = ['상호','대표자','종업원수','전화','팩스','주소','자치구']
# table['자치구'] = list(map(lambda x : x.replace('\t',''),  table['자치구']))
# table.to_csv(file_name, index=None)

table = pd.read_csv(file_name).fillna('')
table.tail(3)

100%|██████████| 641/641 [01:38<00:00,  6.49it/s]


,상호,대표자,종업원수,전화,팩스,주소,자치구
638,(주)심원테크,김준호,9명,02-2627-5650,02-3463-0069,(08590) 서울특별시 금천구 가산동 319 호서대벤처타워 서울특별시 금천구 가...,서울시금천구
639,(주)프레시플라워,국영진,.,02-572-5506,,(06779) 서울특별시 서초구 양재동 323-1 양재2동 주민센터 .,서울시서초구
640,아이티희망나눔(주),김현오,.,02-936-9779,0505-873-1010,(01852) 서울특별시 노원구 동일로191가길 48 (공릉동) 2층,서울시노원구


# **Result Table to XSL**

In [117]:
import pandas as pd

In [124]:
table = pd.read_csv('data/hnm_store.csv').fillna('')
table = table.sort_values(['자치구', '상호']).reset_index(drop=True)
table.to_excel('data/hnm_store.xlsx')
table.tail(3)

,상호,대표자,종업원수,전화,팩스,주소,자치구
638,중랑사랑봉제협동조합,김효영,3,0222699622,0264555464,(02157) 서울특별시 중랑구 면목동 89-34 청풍빌딩 2층,서울시중랑구
639,피제이에스가파치협동조합,박정섭,1,02-436-8446,02-436-8456,(02173) 서울특별시 중랑구 봉우재로70길 9 (망우동) 2층,서울시중랑구
640,하이그린 사회적협동조합,강세호,30,02-1544-7876,070-8622-7877,(02048) 서울특별시 중랑구 봉화산로 123 (상봉동) 신내테크노타운 703호 ...,서울시중랑구


In [125]:
table = pd.read_csv('data/hnm_items.csv')
table = table.sort_values(['업체', '상품']).reset_index(drop=True)
table.to_excel('data/hnm_items.xlsx')
table.tail(3)

,업체,상품,인증,가격,상품코드
12104,힐링필링협동조합,이니셜각인반지,협동조합,"50,000 원",2097
12105,힐링필링협동조합,이니셜볼마커,협동조합,"23,000 원",2088
12106,힐링필링협동조합,책갈피 만들기,협동조합,"10,000 원",2092
